# Group 3 - Coral Bleaching Detection
## JBG060 Capstone Data Challenge 2025-2026 Q1
This file runs the SegFormer - MiT-B2 model on the dataset specified in `root` in the file `config.py`.

After training, inference is run on the test set where evaluation metrics will be calculated and printed in a table format.

Additionally, an image will be sampled from the dataset where a Grad-CAM heatmap is placed over the image.
The heatmap shows the regions that the model found most important for its output.

In [ ]:
# imports
import torch
from image_preprocessing import ImagePreprocessing
from config import __RANDOM_STATE__, root, input_path, output_path
from fine_tune import fine_tune, color_cluster
from inference import inference
from gradcam import grad_cam

In [ ]:
# set seed for reproducibility
torch.manual_seed(__RANDOM_STATE__)
torch.cuda.manual_seed(__RANDOM_STATE__)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

### Perform Image Preprocessing
The code for these functions can be found in `image_preprocessing.py`.

In [ ]:
# preprocess images
processed_images = ImagePreprocessing(input_path, output_path)
processed_images.cluster_images()  # cluster images
processed_images.apply_filters()  # apply CMY filters to each cluster
color, cluster = color_cluster(root)  # to name the checkpoints folder

In [ ]:
# connect to GPU, else use CPU
device_count = torch.cuda.device_count()  # connect to GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
for i in range(device_count):
    print(f"CUDA device {i}: {torch.cuda.get_device_name(i)}")

### Fine-tune the Model
The code can be found in `fine_tune.py`.

In [ ]:
# fine-tune model on dataset specified in `root` variable in `config.py` file
fine_tune(device=device, color=color, cluster=cluster, num_classes=3)

### Obtain Evaluation Metrics on the Test Set
The code can be found in `inference.py`.

In [ ]:
# inference: get evaluation metrics on test set
inference(N_CLASSES=3, device=device)

### Explainability of the Model
The code can be found in `gradcam.py`.

In [ ]:
# run Grad-CAM on a sample image
grad_cam(N_CLASSES=3, DEVICE=device)